In [1]:
import tensorflow as tf
import numpy as np
import scipy.io.wavfile
import os
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [2]:
# data preprocessing
seq_length = 1
merge_factor = 1#temporal resolution = merge_factor/16000 #discrimination_window = merge_factor*seq_length/16000
n_train = 16000; n_test = 1600;
num_channels = 2
input_size = num_channels #n_channels*merge_factor
num_classes = 2 #n_directions
x = tf.placeholder(tf.float32, [None,seq_length, input_size])
labels = tf.placeholder(tf.int32, [None])
offset = 64
train_x = []; train_t = []
test_x = []; test_t = []
for i in range(num_classes):
    temp_data = scipy.io.wavfile.read('Speech/'+'d'+str(0+(180*i))+'.wav')[1][:,::3]
    for j in range(n_train+n_test):
        vec = np.array(temp_data[offset+(j*seq_length*merge_factor):offset+((j+1)*seq_length*merge_factor)])
        vec = vec.reshape(seq_length,merge_factor,2)
        vec = np.mean(vec,axis=1)
        if j%(11) != 5:
            train_x.append(vec)
            train_t.append(i)
    #for j in range(n_train,n_train+n_test):
        else:
            test_x.append(vec)
            test_t.append(i)

train_x = np.array(train_x); train_t = np.array(train_t)
train_x = train_x.reshape(n_train*num_classes,seq_length,input_size)
test_x = np.array(test_x); test_t = np.array(test_t)
test_x = test_x.reshape(n_test*num_classes,seq_length,input_size)

IOError: [Errno 2] No such file or directory: 'noise3/d0.wav'

In [310]:
train_x.shape

(32000, 1, 2)

In [311]:
# hyperparameters
hidden_size = 100
learning_rate = tf.placeholder(tf.float32,[])
lambda_ = tf.placeholder(tf.float32,[])

In [312]:
# parameters
# r_i = np.sqrt(6.0/(input_size + context_size))
# r_c = np.sqrt(6.0/(context_size + context_size))
# r_y = np.sqrt(6.0/(context_size + num_classes))
params = {
    'Wxh': tf.Variable(tf.random_normal([input_size, hidden_size],stddev=0.01)),
    'Whh': tf.Variable(tf.random_normal([hidden_size, hidden_size],stddev = 0.01)),
    'Why': tf.Variable(tf.random_normal([hidden_size,num_classes],stddev = 0.01)),
    'bh': tf.Variable(tf.zeros([hidden_size])),
    'by': tf.Variable(tf.zeros([num_classes]))
}

In [313]:
# architecture
def RNN(x,params):
    batch_size = tf.shape(x)[0]
    h = tf.zeros([batch_size,hidden_size])
    for t in range(seq_length):
        h = tf.tanh(tf.add(tf.add(tf.matmul(x[:,t,:],params['Wxh']),tf.matmul(h,params['Whh'])),params['bh']))
    y = tf.add(tf.matmul(h,params['Why']),params['by'])
    return y

In [314]:
# training strategy
y = RNN(x,params)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(y,labels))# + lambda_*(tf.reduce_sum(tf.mul(params['Wxh'],params['Wxh'])) + tf.reduce_sum(tf.mul(params['Whh'],params['Whh'])) + tf.reduce_sum(tf.mul(params['Why'],params['Why'])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cost)
capped_grads_and_vars = [(tf.clip_by_value(gv[0],-1.,1.), gv[1]) for gv in grads_and_vars]
capped_optimizer = optimizer.apply_gradients(capped_grads_and_vars)
saver = tf.train.Saver()

In [315]:
sess.run(tf.initialize_all_variables())

In [318]:
n_iter = 0
while True:
    _,cost_ = sess.run([capped_optimizer,cost],feed_dict={x:train_x,labels:train_t,learning_rate:0.5,lambda_:0.0})
    if n_iter % 100 == 0:
        print('iter'+str(n_iter)+' cost:'+str(cost_))
        y_train = sess.run(y,feed_dict={x:train_x})
        print 'train pf = ' + str((sum(np.argmax(y_train,axis=1) == train_t) + 0.0)/(n_train*num_classes))
        y_test = sess.run(y,feed_dict={x:test_x})
        print 'test pf = ' + str((sum(np.argmax(y_test,axis=1) == test_t) + 0.0)/(n_test*num_classes))+'\n'
#         mypath = 'vars/iter'+str(n_iter)
#         if not os.path.isdir(mypath):
#            os.makedirs(mypath)
#         myfile = mypath+'/'+str(n_iter)
#         saver.save(sess,myfile,write_meta_graph=False)
    n_iter += 1

iter0 cost:9.43787
train pf = 0.505375
test pf = 0.4821875

iter100 cost:5.60346
train pf = 0.5000625
test pf = 0.4996875

iter200 cost:5.59425
train pf = 0.49990625
test pf = 0.5009375

iter300 cost:5.59126
train pf = 0.49990625
test pf = 0.5009375

iter400 cost:5.59613
train pf = 0.49990625
test pf = 0.5009375

iter500 cost:5.60416
train pf = 0.49990625
test pf = 0.5009375

iter600 cost:5.61221
train pf = 0.49990625
test pf = 0.5009375

iter700 cost:5.61877
train pf = 0.49990625
test pf = 0.5009375

iter800 cost:5.62329
train pf = 0.49990625
test pf = 0.5009375

iter900 cost:5.62581
train pf = 0.49990625
test pf = 0.5009375



KeyboardInterrupt: 

In [299]:
sess.close()